Reference: https://towardsdatascience.com/web-scraping-indeed-com-e5591790736d

Reference:
https://github.com/rowandl/portfolio/blob/master/Webscraping%20Indeed.com/Webscraping%20Indeed.ipynb

In [2]:
#!pip install pandas
#!pip install bs4
#!pip install matplotlib
#!pip install lxml

In [3]:
import pandas as pd
import urllib
from bs4 import BeautifulSoup
import csv
import time

from IPython.display import HTML

In [ ]:
# top 10 canada cities
cities = ['Toronto', 'Montreal', 'Vancouver', 'Calgary', 'Edmonton', 'Ottawa', 'Winnipeg', 'Quebec+City', 'Hamilton', 'Kitchener']
#cities = ['Toronto']

# Set this to a high-value to generate more results. High value might get block by website
max_results_per_city = 100
#max_results_per_city = 10

#making a dataframe which collects all the information from my webscraping
#https://www.indeed.ca/jobs?q=data+scientist+%2420%2C000&l=Toronto&start=0
url_template = "https://www.indeed.ca/jobs?q=data+scientist+%2420%2C000&l={}&start={}"
base_url = "https://ca.indeed.com"

df = pd.DataFrame(columns=["location", 'company', 'job_title', 'salary', 'job_description','job_url'])

for city in cities:
    for start in range(0, max_results_per_city, 10):
        url = url_template.format(city,start)
        html = urllib.request.urlopen(url).read()
        soup = BeautifulSoup(html, "html.parser")
        for b in soup.find_all('div', attrs = {'class': 'result'}):
            try:
                location = b.find('div', attrs = {'class': 'location'}).text
            except:
                try:
                    location = b.find('span', attrs = {'class': 'location'}).text
                except:
                    location = 'NA'

            job_title = b.find('a', attrs = {'data-tn-element':'jobTitle'}).text
            job_title = job_title.replace("\n","")
            
            job_url = b.find('a', attrs = {'data-tn-element':'jobTitle'}).get('href')

            try:
                company = b.find('span', attrs = {'class':'company'}).text
                company = company.replace("\n","")
            except:
                company = 'NA'
            try:
                salary = b.find('td', attrs = {'class' : 'snip'}).find('nobr').text  
            except:
                salary = 'NA'
            
            time.sleep(2) # to prevent web blocking

            url_job = 'https://ca.indeed.com' + job_url
            html_job = urllib.request.urlopen(url_job).read()
            soup_job = BeautifulSoup(html_job, "html.parser")

            try:
                description = soup_job.find('div', attrs = {'id': 'jobDescriptionText'}).text
            except:
                description = 'NA'
            
            print("%s: %s > %s > %s" % (city, location, company, job_title))

            if (description != 'NA'):
                df = df.append({"location":location,"company":company, "job_title": job_title, "salary": salary, "job_description": description, "job_url": url_job}, ignore_index=True)
            
            time.sleep(3) # to prevent web blocking
   
df.drop_duplicates(inplace=True)
df.company.replace(inplace=True,regex=True,to_replace="\n",value="")
df.job_title.replace(inplace=True,regex=True,to_replace="\n",value="")
df.head()

Toronto: Etobicoke, ON > Relay Medical Corp. > Data Science / Machine Vision Intern
Toronto: Greater Toronto Area, ON > fgf brands > Data Scientist Co-op ( 4 or 8 Month Winter Placement)
Toronto: Toronto, ON > Indigo Park > Data Scientist
Toronto: Toronto, ON > QuadReal > Intern, Data Scientist
Toronto: Toronto, ON > TELUS > Data Scientist
Toronto: North York, ON > Poros Career > Data Analyst
Toronto: Toronto, ON > Equifax > Data Scientist - Client Success
Toronto: Toronto, ON > Overbond > Data Scientist
Toronto: Toronto, ON > TELUS > Data Scientist - TELUS Digital
Toronto: Toronto, ON > Sunnyfuture Group > Data Analyst/Financial Analyst
Toronto: Toronto, ON > float simple > Lead Data Scientist - BI Analytics
Toronto: Mississauga, ON > Compass Digital Labs > Data Scientist (Decision Science)
Toronto: Mississauga, ON > Hatch > Data Scientist
Toronto: Toronto, ON > TripStack > Data Scientist
Toronto: Toronto, ON > Zynga > Data Scientist II
Toronto: Toronto, ON > Overbond > Data Scientist

# **Save to CSV**

In [52]:
from datetime import datetime
from google.colab import files

# sending it to csvs to save the data
now = datetime.now()
dt_string = now.strftime("%y%m%d%H%M")
filename = "Indeed_" + dt_string + ".csv"
df.to_csv(filename , sep=',', encoding='utf-8')

files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>